In [ ]:
# |default_exp whisper

# Constantes

In [ ]:
# |export

AUDIO_PATH = "audios"

# list mp3 files

In [ ]:
# |export

from mongo_episode import get_audio_path
import os, glob
from typing import List, Optional


def list_mp3_files(
    audio_path=AUDIO_PATH, sort_by_size: Optional[str] = None
) -> List[str]:
    """
    Liste tous les fichiers MP3 dans le répertoire spécifié.
    Args:
        audio_path (str): Le chemin du répertoire contenant les fichiers audio. Par défaut, utilise la constante AUDIO_PATH.
        sort_by_size (Optional[str]): 'asc' pour trier par taille croissante, 'desc' pour décroissante, None pour aucun tri.
    Returns:
        list: Une liste des chemins de fichiers MP3 trouvés, triés si demandé.
    """
    fullpath = get_audio_path(audio_path, year="")
    files = glob.glob(os.path.join(fullpath, "**/*.mp3"), recursive=True)

    if sort_by_size:
        order = sort_by_size.lower()
        if order not in ("asc", "desc"):
            raise ValueError("sort_by_size must be 'asc', 'desc' or None")
        files.sort(key=lambda p: os.path.getsize(p), reverse=(order == "desc"))

    return files


def list_audio_files(
    audio_path=AUDIO_PATH, sort_by_size: Optional[str] = None
) -> List[str]:
    """
    Liste tous les fichiers audio (MP3 et M4A) dans le répertoire spécifié.

    Args:
        audio_path (str): Le chemin du répertoire contenant les fichiers audio. Par défaut, utilise la constante AUDIO_PATH.
        sort_by_size (Optional[str]): 'asc' pour trier par taille croissante, 'desc' pour décroissante, None pour aucun tri.

    Returns:
        list: Une liste des chemins de fichiers audio (MP3 et M4A) trouvés, triés si demandé.
    """
    fullpath = get_audio_path(audio_path, year="")

    mp3_files = glob.glob(os.path.join(fullpath, "**/*.mp3"), recursive=True)
    m4a_files = glob.glob(os.path.join(fullpath, "**/*.m4a"), recursive=True)

    files = mp3_files + m4a_files

    if sort_by_size:
        order = sort_by_size.lower()
        if order not in ("asc", "desc"):
            raise ValueError("sort_by_size must be 'asc', 'desc' or None")
        files.sort(key=lambda p: os.path.getsize(p), reverse=(order == "desc"))

    return files

In [ ]:
list_mp3_files(sort_by_size="asc")[:5]

['/workspaces/lmelp/audios/2016/14007-27.11.2016-ITEMA_21148290-0.mp3',
 '/workspaces/lmelp/audios/2024/14007-10.11.2024-ITEMA_23920569-2024F4007S0315-22.mp3',
 '/workspaces/lmelp/audios/2025/14007-14.09.2025-ITEMA_24246092-2025F4007S0257-NET_MFI_1A3CF54C-C3D3-4ABF-805B-2B3A8E4ED735-22.mp3',
 '/workspaces/lmelp/audios/2024/14007-01.12.2024-ITEMA_23942372-2024F4007S0336-22.mp3',
 '/workspaces/lmelp/audios/2025/14007-07.09.2025-ITEMA_24238506-2025F4007S0250-NET_MFI_362EAAE4-F232-40E7-9F69-12467ED0D96A-22.mp3']

In [ ]:
list_audio_files(sort_by_size="desc")[-5:]

['/workspaces/lmelp/audios/1969/14007-30.11.1969-ITEMA_23787926-2024F4007E0117-27.m4a',
 '/workspaces/lmelp/audios/1996/14007-08.12.1996-ITEMA_23787640-2024F4007E0090-27.m4a',
 '/workspaces/lmelp/audios/1960/14007-15.12.1960-ITEMA_23787917-2024F4007E0110-27.m4a',
 '/workspaces/lmelp/audios/1984/14007-11.11.1984-ITEMA_23787854-2024F4007E0102-27.m4a',
 '/workspaces/lmelp/audios/1992/14007-27.09.1992-ITEMA_23787897-2024F4007E0094-27.m4a']

# extract whisper

In [ ]:
# |export

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


def extract_whisper(audio_filename: str) -> str:
    """
    Extrait la transcription d'un fichier audio en utilisant le modèle Whisper.

    Args:
        audio_filename (str): Le chemin du fichier audio à transcrire.

    Returns:
        str: La transcription du fichier audio.
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    generate_kwargs = {
        "language": "fr",
    }

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
        chunk_length_s=30,
        batch_size=16,  # batch size for inference - set based on your device
        generate_kwargs=generate_kwargs,
    )

    result = pipe(
        audio_filename,
        return_timestamps=True,
        ignore_warning=True,
    )

    return result["text"]

In [ ]:
petit_audio = list_audio_files(sort_by_size="asc")[0]

print("=" * 50)
print(f"filename: {petit_audio} ")

whisper = extract_whisper(petit_audio)

whisper[:150]

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


" France Inter C'était très bien accueilli aussi par la presse dès le début de la saison C'est le livre de Patrick Chamoiseau chez Gallimard, Texaco Ch"

# Extract whisper long

In [ ]:
# |export

from pydub import AudioSegment
import tempfile
import os
import soundfile as sf
import torch


def extract_whisper_long(
    audio_filename: str, chunk_s: int = 30, overlap_s: int = 1
) -> str:
    """
    Splits long audio into overlapping chunks, preprocesses each chunk with the processor
    (using return_attention_mask) and runs model.generate for each chunk, then returns concatenated text.

    Important: ensure audio chunks are sampled at the feature extractor's expected rate (16kHz).
    This implementation exports each chunk to WAV at 16000 Hz to avoid the sampling-rate ValueError.
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    model_id = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)
    processor = AutoProcessor.from_pretrained(model_id)

    generate_kwargs = {"language": "fr"}

    seg = AudioSegment.from_file(audio_filename)
    dur_ms = len(seg)
    step_ms = (chunk_s - overlap_s) * 1000
    chunk_ms = chunk_s * 1000
    texts = []

    for start in range(0, max(1, dur_ms), int(step_ms)):
        end = start + chunk_ms
        piece = seg[start:end]
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
            tmp_path = tmp.name
            # Export the chunk as WAV forced to 16 kHz to match WhisperFeatureExtractor expectations
            # requires ffmpeg (used by pydub) available in the container.
            piece.export(tmp_path, format="wav", parameters=["-ar", "16000"])
        try:
            # read chunk as numpy array (float) + sr
            speech, sr = sf.read(tmp_path)
            if sr != processor.feature_extractor.sampling_rate:
                raise RuntimeError(
                    f"Unexpected sampling rate {sr}, expected {processor.feature_extractor.sampling_rate}"
                )

            # processor -> return_tensors="pt" and return_attention_mask to follow deprecation guidance
            inputs = processor(
                speech,
                sampling_rate=sr,
                return_tensors="pt",
                return_attention_mask=True,
            )
            # support both possible keys
            feat = inputs.get("input_features", inputs.get("input_values"))
            if feat is None:
                raise RuntimeError("Processor did not return input features")
            feat = feat.to(device).to(dtype=torch_dtype)

            # run generate on model
            generated = model.generate(feat, **generate_kwargs)
            decoded = processor.tokenizer.batch_decode(
                generated, skip_special_tokens=True
            )
            texts.append(decoded[0].strip() if decoded else "")
        finally:
            os.remove(tmp_path)
        if end >= dur_ms:
            break

    return " ".join(t for t in texts if t)

In [ ]:
petit_audio = list_audio_files(sort_by_size="asc")[0]

print("=" * 50)
print(f"filename: {petit_audio} ")

whisper_long = extract_whisper_long(petit_audio)

whisper_long[:150]

filename: /workspaces/lmelp/audios/1992/14007-27.09.1992-ITEMA_23787897-2024F4007E0094-27.m4a 


: 

In [ ]:
whisper

"C'était très bien accueilli aussi par la presse dès le début de la saison, c'est le livre de Patrick Chamoiseau chez Gallimard, Texaco. Chamoiseau qui est un des jeunes écrivains antillais qui s'est fait connaître depuis quelques années avec de très beaux livres et qui là c'est vrai a les honneurs de toute la presse depuis quelques semaines. Alors qui a lu Texaco ici? Sophie Chérez, vous avez lu? Personne, comme tout le monde ne l'a parlé, personne. On ne l'est pas au programme. On ne l'est pas au programme, c'est comme à la rentrée scolaire. c'est comme à la rentrée scolaire non plus, Jean-Lidier? Moi je l'ai lu. Josiane? Oui, moi je l'ai lu, je l'ai défendu, je trouve ça très très beau. C'est un livre en effet un peu lourd comme Jean-François, je pense qu'il trouverait ça, c'est assez touffu, c'est un gros livre. Mais ce que j'aime beaucoup chez Chamoiseau, c'est la manière dont il utilise la langue française, c'est pas du créole, c'est un français qui l'a, comme dit Milan Kundera, 

# store whisper in db 

In [ ]:
# |export

from bson import ObjectId
import pymongo


def store_whisper_in_db(
    whisper: str,
    collection: pymongo.collection.Collection,
    oid: str,
    force: bool = False,
    verbose: bool = False,
) -> bool:
    """
    Stocke la transcription Whisper dans la base de données.

    Args:
        whisper (str): La transcription du fichier audio.
        collection: La collection pymongo.
        oid (str): L'identifiant de l'épisode.
        force (bool, optional): Si True, écrase le Whisper existant. Par défaut, False.
        verbose (bool, optional): Si True, affiche des messages détaillés. Par défaut, False.

    Returns:
        bool: True si le Whisper a été stocké, False sinon.
    """
    # Récupération du document
    document_entry = collection.find_one({"_id": ObjectId(oid)})

    if document_entry is None:
        if verbose:
            print(f"Document avec l'oid {oid} non trouvé")
        return False

    if "whisper" in document_entry and not force:
        if verbose:
            print(
                f"Whisper déjà stocké pour l'oid {oid}, et on ne force pas le stockage"
            )
        return False
    else:
        document_entry["whisper"] = whisper
        collection.update_one({"_id": ObjectId(oid)}, {"$set": document_entry})
        if verbose:
            print(f"Whisper stocké pour l'oid {oid}")
        return True

In [ ]:
from mongo import get_collection

col = get_collection()
oid = "6773e32258fc5717f3516b98"
store_whisper_in_db("test whisper", col, oid, force=True, verbose=True)

Whisper stocké pour l'oid 6773e32258fc5717f3516b98


True

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("09 whisper mp3.ipynb", ".")